# Mobility Analysis

**Note: DFI Queries Will Not Work**

**The Data Flow Index server used for this workshop is no longer running.  The workshop materials are left up _as is_ but queries will not run.  If you would like to trial the Data Flow Index please reach out to General System at [https://www.generalsystem.com/contact-us](https://www.generalsystem.com/contact-us).**

In [ ]:
import json
import shutil
from pathlib import Path

import altair as alt
import geopandas as gpd
import pandas as pd
import urllib3

alt.data_transformers.disable_max_rows()

## 0. Utility Functions

In [ ]:
# Define the theme by returning the dictionary of configurations
FONT = "Space Grotesk"


def gs_theme():
    return {
        "config": {
            "range": {
                "category": [
                    "#FF5722",
                    "#03A9F4",
                    "#F44336",
                    "#00BCD4",
                    "#4CAF50",
                    "#FFEB3B",
                    "#9C27B0",
                    "#E91E63",
                    "#795548",
                    "#9E9E9E",
                ],
                "heatmap": {"scheme": ["#FFFFFF00", "#FF5008FF"]},
                "ordinal": {"scheme": ["#FFFFFF00", "#FF5008FF"]},
                "ramp": {"scheme": ["#FFFFFF88", "#FF5008FF"]},
                "diverging": {"scheme": ["#FFFFFF00", "#FF5008FF"]},
            },
            "axis": {
                "labelFont": FONT,
                "titleFont": FONT,
            },
            "legend": {
                "labelFont": FONT,
                "titleFont": FONT,
            },
            "title": {
                "font": FONT,
                "subtitleFont": FONT,
            },
            "mark": {
                "font": FONT,
            },
            "header": {
                "labelFont": FONT,
                "titleFont": FONT,
            },
        },
    }


# Register the custom theme under a chosen name
alt.themes.register("gs_theme", gs_theme)

# Enable the newly registered theme
alt.themes.enable("gs_theme")


def load_location_data(filename: str, url: str) -> gpd.GeoDataFrame:
    """ "Downloads the file at url and saves to a file called filename, returns gdf
    e.g. url = "https://d3ftlhu7xfb8rb.cloudfront.net/blank_street_coffees.geoparquet"
    """
    Path(filename).parent.mkdir(parents=True, exist_ok=True)
    http = urllib3.PoolManager()
    with open(filename, "wb") as out:
        r = http.request("GET", url, preload_content=False)
        shutil.copyfileobj(r, out)

    return gpd.read_parquet(filename)

## I. Finding Blank Street Coffee Customers

### A. Get the data

We will be loading in data from a public S3 bucket using cloudfront. The customer dataset was collected using the dfi queries in the previous section. It is loaded here directly in the interest of expediency - we only have 1 hour for the workshop!

In [ ]:
# Read customer dataset from cloudfront url
records_df = pd.read_parquet("https://d3ftlhu7xfb8rb.cloudfront.net/mobility-analysis-input-merged.parquet").rename(
    columns={"uuid": "customer_id"}
)

Of course, for this mobility analysis, we will also need the relevant OSM building data.

In [ ]:
# Load in OSM building data
osm_gdf = load_location_data("osm_gdf", "https://d3ftlhu7xfb8rb.cloudfront.net/london_nyc_osm.geoparquet")
osm_ids = osm_gdf["osm_id"]

# Load in Blank Street Coffee Location dataset
bsc_gdf = load_location_data(
    "bsc_gdf",
    "https://d3ftlhu7xfb8rb.cloudfront.net/blank_street_coffee_callsigns.geoparquet",
)
bsc_osm_ids = bsc_gdf["osm_id"]

### B. Analysis by BSC Location

In [ ]:
# Group records by dwell
agg_df = (
    records_df[records_df["transportation_mode"] == "dwelling"]
    .groupby(by=["customer_id", "route"], as_index=False)
    .agg(
        start_time=("timestamp", "min"),
        end_time=("timestamp", "max"),
        location_id=("start_location_id", "first"),
    )
)
bsc_dwells_df = agg_df[agg_df["location_id"].isin(bsc_osm_ids)]

#### 1. Number of visits

In [ ]:
location_popularity = (
    bsc_dwells_df.groupby("location_id")
    .agg(dwell_count=("route", "count"))
    .reset_index()
    .merge(
        bsc_gdf[["osm_id", "callsign"]],
        left_on="location_id",
        right_on="osm_id",
        how="left",
    )
    .drop("osm_id", axis=1)
    .sort_values(by="dwell_count", ascending=False)
    .reset_index(drop=True)
)
location_popularity.head()

#### 2. Distribution of customers

In [ ]:
location_customers = (
    bsc_dwells_df.groupby("location_id")["customer_id"]
    .nunique()
    .reset_index()
    .merge(
        bsc_gdf[["osm_id", "callsign"]],
        left_on="location_id",
        right_on="osm_id",
        how="left",
    )
    .drop("osm_id", axis=1)
    .rename(columns={"customer_id": "customer_count"})
    .sort_values(by="customer_count", ascending=False)
    .reset_index(drop=True)
)

location_customers.head()

In [ ]:
# Bar chart of number of customers per BSC location
customers_per_location = (
    alt.Chart(location_customers)
    .mark_bar(cornerRadiusTopLeft=3, cornerRadiusTopRight=3)
    .encode(
        x=alt.X(
            "callsign:N",
            title="BSC Locations",
            sort="-y",
            axis=alt.Axis(labelAngle=-45),
        ),
        y=alt.Y("customer_count:Q", title="Number of Customers", axis=alt.Axis(format="d")),
        color=alt.value("#03A9F4"),
    )
    .properties(width=800, height=300, title="BSC Location Popularity")
)

customers_per_location

#### 3. Heatmap of visits by time period

In [ ]:
days_of_week = [
    "Monday",
    "Tuesday",
    "Wednesday",
    "Thursday",
    "Friday",
    "Saturday",
    "Sunday",
]

bsc_dwells_df = bsc_dwells_df.assign(
    day_of_week=lambda df: df.start_time.dt.day_name(),
    hour_of_day=lambda df: df.start_time.dt.hour,
)

all_combinations = pd.MultiIndex.from_product([days_of_week, range(24)], names=["day_of_week", "hour_of_day"])
heatmap_df = (
    bsc_dwells_df.groupby(["day_of_week", "hour_of_day"])
    .size()
    .reset_index(name="count")
    .set_index(["day_of_week", "hour_of_day"])
    .reindex(all_combinations)
    .reset_index()
    .fillna(value={"count": 0})
)
heatmap_df

In [ ]:
heatmap = (
    alt.Chart(heatmap_df)
    .mark_rect(cornerRadius=5)
    .encode(
        alt.X("hour_of_day:O").title("Hour of Day"),
        alt.Y("day_of_week:O", sort=days_of_week).title("Day of Week"),
        alt.Color("count:Q").title("Number of Dwells"),
    )
    .properties(title="Blank Street Coffee shops are busiest between 7-9am")
)

heatmap

## II. Cross Visitation

### A. Where has one particular customer dwelled?

In [ ]:
# Choose a customer
customer = "65b753d2-b523-467f-9c39-bc0fd6e2393b"

customer_dwells_df = agg_df[agg_df["customer_id"] == customer].reset_index(drop=True)

Since the osm building data is messy and not every building has a name or category, let's make a label column to clear things up as much as possible

In [ ]:
def create_label_column(df: pd.DataFrame, id_column: str = "location_id"):
    """
    Determine the label for each row based on the amount of metadata present
    """

    def label(row):
        if row[id_column] in bsc_gdf["osm_id"].values:
            callsign = bsc_gdf.loc[bsc_gdf["osm_id"] == row[id_column], "callsign"].values[0]
            return f"BSC - {callsign}"
        elif pd.notna(row["name"]) and pd.notna(row["category"]):
            return f'{row["name"]} - {row["category"]} - id_{row[id_column]}'
        elif pd.notna(row["name"]):
            return f'{row["name"]} - id_{row[id_column]}'
        elif pd.notna(row["category"]):
            return f'{row["category"]} - id_{row[id_column]}'
        else:
            return f"id_{row[id_column]}"

    # Apply the label function to each row
    df["label"] = df.apply(label, axis=1)

    return df


def merge_osm_data(dwells_df: pd.DataFrame, osm_gdf: gpd.GeoDataFrame):
    """Merge OSM building data onto dwells dataframe and create labels"""
    dwells_df = (
        pd.merge(
            dwells_df,
            osm_gdf[["osm_id", "name", "category"]],
            left_on="location_id",
            right_on="osm_id",
            how="left",
        )
        .drop(columns=["osm_id"])
        .pipe(create_label_column)
    )
    return dwells_df

In [ ]:
cd_df = merge_osm_data(customer_dwells_df, osm_gdf)
location_dwell_count = pd.DataFrame(cd_df["label"].value_counts().reset_index())
location_dwell_count = location_dwell_count.loc[~location_dwell_count["label"].str.startswith("id_")]

In [ ]:
# Bar chart of number of dwells per building for the customer
dwells_per_location = (
    alt.Chart(location_dwell_count)
    .mark_bar(cornerRadiusTopLeft=3, cornerRadiusTopRight=3)
    .encode(
        x=alt.X("label:N", title="Buildings", sort="-y", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("count:Q", title="Number of Dwells", axis=alt.Axis(format="d")),
        color=alt.value("#03A9F4"),
    )
    .configure_axis(labelLimit=100)
    .properties(title="This customer visited 'BSC - Kopi Tubruk' 4 times")
)

dwells_per_location

### B. Finding aggregated dwell information from the customers

Of all the people that visit a (the most popular for this case) BSC location, where else do they go

In [ ]:
agg_cd_df = merge_osm_data(agg_df, osm_gdf)

In [ ]:
label_counts = agg_cd_df["label"].value_counts().reset_index()
label_counts.columns = ["label", "frequency"]

agg_cd_df["dwell_time"] = ((agg_cd_df["end_time"] - agg_cd_df["start_time"]).dt.total_seconds() / 3600).astype(float)
dwell_times = agg_cd_df.groupby("label").agg(total_time=("dwell_time", "sum"))

labels_and_times = label_counts.merge(dwell_times, on="label")
labels_and_times

In [ ]:
cross_visitation = (
    alt.Chart(labels_and_times.head(20))
    .mark_bar(cornerRadiusTopLeft=3, cornerRadiusTopRight=3)
    .encode(
        x=alt.X("label:O", title="Buildings", sort="-y", axis=alt.Axis(labelAngle=-45)),
        y=alt.Y("frequency:Q", title="Number of Dwells"),
        color=alt.value("#03A9F4"),
        tooltip="total_time",
    )
    .properties(title="20 Most Popular Buildings")
    .configure_legend(disable=True)
)
cross_visitation.save("../presentation/src/assets/figures/cross_visitation.html")
cross_visitation

## III. Hotspots in Time

In [ ]:
agg_cd_df["day_of_week"] = agg_cd_df["start_time"].dt.day_name()
agg_cd_df["hour_of_day"] = agg_cd_df["start_time"].dt.hour

time_period_breakdown = (
    agg_cd_df.assign(day_of_week=pd.Categorical(agg_cd_df["day_of_week"], categories=days_of_week, ordered=True))
    .groupby(["day_of_week", "hour_of_day"])["label"]
    .agg(
        lambda x: pd.Series(
            {
                "mode_value": x.mode().iloc[0],
                "mode_count": (x == x.mode().iloc[0]).sum(),
            }
        )
    )
    .reset_index()
)

time_period_breakdown[["mode_value", "mode_count"]] = time_period_breakdown["label"].apply(pd.Series)
time_period_breakdown.drop(columns=["label"], inplace=True)

time_period_breakdown = (
    time_period_breakdown.set_index(["day_of_week", "hour_of_day"]).reindex(all_combinations).reset_index()
)

time_period_breakdown["mode_count"] = time_period_breakdown["mode_count"].fillna(0).astype(int)

time_period_breakdown

In [ ]:
time_period_heatmap = (
    alt.Chart(time_period_breakdown)
    .mark_rect(cornerRadius=5)
    .encode(
        alt.X("hour_of_day:O").title("Hour of Day"),
        alt.Y("day_of_week:O", sort=days_of_week).title("Day of Week"),
        alt.Color("mode_count:Q").title("Number of Dwells"),
        tooltip="mode_value",
    )
    .properties(title="At 12pm on a Wednesday, customers' most popular location is VERG Brooklyn")
)

time_period_heatmap.save("../presentation/src/assets/figures/time_period_heatmap.html")
time_period_heatmap

## IV. Customer Journey

In [ ]:
results = []
final_dwell = customer_dwells_df.index.max()

# Filter dwells for BSC location dwells
bsc_dwells_df = customer_dwells_df[customer_dwells_df["location_id"].isin(bsc_osm_ids)]

# For each of the bsc locations the customer visited...
for bsc_location in bsc_dwells_df["location_id"]:
    # ...find which dwells are at each location and...
    bsc_dwell_indexes = customer_dwells_df[customer_dwells_df["location_id"] == bsc_location].index.unique()

    # ...loop through dwells to find the osm ids of the previous and next dwells
    for index in bsc_dwell_indexes:
        row = {"bsc_location": bsc_location}
        if index != final_dwell:
            row["next_location"] = customer_dwells_df[customer_dwells_df.index == index + 1]["location_id"].unique()[0]
        else:
            row["next_location"] = None

        if index != 0:
            row["previous_location"] = customer_dwells_df[customer_dwells_df.index == index - 1][
                "location_id"
            ].unique()[0]
        else:
            row["previous_location"] = None
        results.append(row)

results_df = pd.DataFrame.from_records(results)

In [ ]:
# This commented out code can be used to create labels for the entire osm dataset
# osm_labels = create_label_column(osm_gdf, id_column="osm_id")
# osm_labels_map = {row["osm_id"]: row["label"] for _, row in osm_labels.iterrows()}

# with open("../presentation/src/assets/osm_map.json","w") as file:
#     json.dump(osm_labels_map, file)

# Attach labels to results using a map
with open("../presentation/src/assets/osm_map.json", "r") as file:
    osm_labels_map = json.load(file)

for i in results_df.columns:
    results_df[i] = results_df[i].apply(lambda x: osm_labels_map[x])

In [ ]:
results_df.head()

In [ ]:
results_df.groupby("bsc_location")[["previous_location", "next_location"]].value_counts().reset_index()[
    ["previous_location", "bsc_location", "next_location", "count"]
]